# Financial Dataset Exploration & Research Feasibility Analysis

**Objective:** Profile the dataset and determine which tier-1 research questions are feasible.

## Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")

In [ ]:
# Define data path
DATA_PATH = Path('/content/drive/MyDrive/Paper_2')

# List files
print("Files in data directory:")
for f in DATA_PATH.iterdir():
    print(f"  {f.name}")

## 1. Load and Inspect Raw Data

In [ ]:
# Load both Excel files
df1 = pd.read_excel(DATA_PATH / 'Data_1.xlsx')
df2 = pd.read_excel(DATA_PATH / 'Data_2.xlsx')

print(f"Data_1.xlsx shape: {df1.shape}")
print(f"Data_2.xlsx shape: {df2.shape}")
print(f"\nTotal columns across both files: {df1.shape[1] + df2.shape[1]}")

In [ ]:
# Preview Data_1
print("=" * 80)
print("DATA_1.xlsx - First 5 rows")
print("=" * 80)
display(df1.head())
print(f"\nColumns ({len(df1.columns)}):")
print(df1.columns.tolist())

In [ ]:
# Preview Data_2
print("=" * 80)
print("DATA_2.xlsx - First 5 rows")
print("=" * 80)
display(df2.head())
print(f"\nColumns ({len(df2.columns)}):")
print(df2.columns.tolist())

## 2. Column Classification

Based on the data dictionary provided:
- **Identifiers (1-5):** Company Name, Exchange:Ticker, Geographic Locations, Company Type, Industry Classifications
- **Market Cap:** 10 annual snapshots (Latest to Latest-9 Years)
- **Profitability:** Gross Profit, Operating Income, EBITDA, Net Income (10 quarterly LTM)
- **Expenses:** R&D, Interest Expense (10 quarterly LTM)
- **Balance Sheet:** Cash, Receivables, Inventory, Current Assets, PP&E, Total Assets, Debt (10 quarterly)
- **Cash Flow:** CapEx, Acquisitions (10 quarterly LTM)
- **Ratios:** ROA%, ROE%, Debt/Equity (10 quarterly)
- **Other:** Employees, Revenue (10 quarterly)

In [ ]:
def classify_columns(columns):
    """Classify columns by type based on naming patterns."""
    classification = {
        'identifiers': [],
        'market_cap': [],
        'profitability': [],
        'expenses': [],
        'balance_sheet': [],
        'cash_flow': [],
        'ratios': [],
        'other_metrics': [],
        'unknown': []
    }
    
    identifier_keywords = ['name', 'ticker', 'exchange', 'location', 'type', 'industry', 'sector', 'country']
    market_cap_keywords = ['market cap', 'marketcap', 'market_cap', 'mkt cap']
    profitability_keywords = ['gross profit', 'operating income', 'ebitda', 'net income', 'ebit']
    expense_keywords = ['r&d', 'research', 'interest expense', 'sga', 'selling']
    balance_sheet_keywords = ['cash', 'receivable', 'inventory', 'current asset', 'ppe', 'pp&e', 
                              'total asset', 'current debt', 'long-term debt', 'lt debt', 'total debt']
    cash_flow_keywords = ['capex', 'capital expenditure', 'acquisition', 'fcf', 'free cash']
    ratio_keywords = ['roa', 'roe', 'debt/equity', 'debt to equity', 'margin', 'ratio']
    other_keywords = ['employee', 'revenue', 'sales', 'headcount']
    
    for col in columns:
        col_lower = col.lower()
        
        if any(kw in col_lower for kw in identifier_keywords):
            classification['identifiers'].append(col)
        elif any(kw in col_lower for kw in market_cap_keywords):
            classification['market_cap'].append(col)
        elif any(kw in col_lower for kw in profitability_keywords):
            classification['profitability'].append(col)
        elif any(kw in col_lower for kw in expense_keywords):
            classification['expenses'].append(col)
        elif any(kw in col_lower for kw in balance_sheet_keywords):
            classification['balance_sheet'].append(col)
        elif any(kw in col_lower for kw in cash_flow_keywords):
            classification['cash_flow'].append(col)
        elif any(kw in col_lower for kw in ratio_keywords):
            classification['ratios'].append(col)
        elif any(kw in col_lower for kw in other_keywords):
            classification['other_metrics'].append(col)
        else:
            classification['unknown'].append(col)
    
    return classification

# Classify columns from both dataframes
all_columns = list(df1.columns) + list(df2.columns)
classified = classify_columns(all_columns)

print("Column Classification Summary:")
print("=" * 60)
for category, cols in classified.items():
    print(f"\n{category.upper()} ({len(cols)} columns):")
    if cols:
        for col in cols[:5]:  # Show first 5
            print(f"  - {col}")
        if len(cols) > 5:
            print(f"  ... and {len(cols) - 5} more")

## 3. Time Period Identification

**Critical for research design:** We need to identify the exact time coverage.

In [ ]:
def extract_time_info(columns):
    """Extract time period information from column names."""
    import re
    
    time_patterns = {
        'annual': r'Latest\s*-?\s*(\d+)\s*Year',
        'ltm': r'LTM\s*-?\s*(\d+)',
        'quarterly': r'Latest\s*Quarter\s*-?\s*(\d+)',
        'latest': r'^.*Latest(?!.*-).*$'
    }
    
    time_info = {'annual_range': [], 'ltm_range': [], 'quarterly_range': []}
    
    for col in columns:
        # Check for annual
        match = re.search(time_patterns['annual'], col, re.IGNORECASE)
        if match:
            time_info['annual_range'].append(int(match.group(1)))
        
        # Check for LTM
        match = re.search(time_patterns['ltm'], col, re.IGNORECASE)
        if match:
            time_info['ltm_range'].append(int(match.group(1)))
        
        # Check for quarterly
        match = re.search(time_patterns['quarterly'], col, re.IGNORECASE)
        if match:
            time_info['quarterly_range'].append(int(match.group(1)))
    
    return time_info

time_info = extract_time_info(all_columns)

print("Time Coverage Analysis:")
print("=" * 60)
if time_info['annual_range']:
    print(f"Annual data: Latest to Latest - {max(time_info['annual_range'])} Years")
if time_info['ltm_range']:
    print(f"LTM data: LTM to LTM - {max(time_info['ltm_range'])} quarters ({max(time_info['ltm_range'])//4} years)")
if time_info['quarterly_range']:
    print(f"Quarterly data: Latest Quarter to Latest Quarter - {max(time_info['quarterly_range'])} ({max(time_info['quarterly_range'])//4} years)")

## 4. Sample Size & Data Quality

In [ ]:
# Check for common identifier to merge
print("Looking for merge keys between Data_1 and Data_2...")
common_cols = set(df1.columns) & set(df2.columns)
print(f"Common columns: {common_cols}")

# If there are common columns, try to merge
if common_cols:
    merge_key = list(common_cols)[0]
    print(f"\nMerging on: {merge_key}")
    df = pd.merge(df1, df2, on=list(common_cols), how='outer')
    print(f"Merged dataset shape: {df.shape}")
else:
    # Check if they have same number of rows (might be column split)
    if len(df1) == len(df2):
        print("\nSame number of rows - concatenating horizontally")
        df = pd.concat([df1, df2], axis=1)
        print(f"Combined dataset shape: {df.shape}")
    else:
        print("\nDifferent row counts - keeping separate for now")
        df = df1  # Use df1 as primary

In [ ]:
# Basic statistics
print(f"\nDataset Overview:")
print("=" * 60)
print(f"Number of companies: {len(df)}")
print(f"Number of variables: {len(df.columns)}")
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Missing value analysis
def missing_analysis(dataframe):
    """Analyze missing values in the dataset."""
    missing = dataframe.isnull().sum()
    missing_pct = (missing / len(dataframe)) * 100
    
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing Count': missing.values,
        'Missing %': missing_pct.values
    })
    
    missing_df = missing_df[missing_df['Missing Count'] > 0]
    missing_df = missing_df.sort_values('Missing %', ascending=False)
    
    return missing_df

missing_df = missing_analysis(df)
print(f"\nColumns with missing values: {len(missing_df)} out of {len(df.columns)}")
print("\nTop 20 columns by missing %:")
display(missing_df.head(20))

In [ ]:
# Visualize missing data pattern
fig, ax = plt.subplots(figsize=(14, 6))

# Sample columns for visualization (every 10th column)
sample_cols = df.columns[::max(1, len(df.columns)//30)]
missing_pct = df[sample_cols].isnull().mean() * 100

missing_pct.plot(kind='bar', ax=ax, color='steelblue', alpha=0.7)
ax.set_ylabel('Missing %')
ax.set_title('Missing Data Pattern (Sampled Columns)')
ax.tick_params(axis='x', rotation=90)
plt.tight_layout()
plt.show()

## 5. Industry & Geographic Distribution

In [ ]:
# Find identifier columns
id_cols = classified['identifiers']
print(f"Identifier columns found: {id_cols}")

# Try to identify industry and geography columns
for col in id_cols:
    print(f"\n{col} - Unique values: {df[col].nunique()}")
    if df[col].nunique() <= 50:  # Show distribution if reasonable
        print(df[col].value_counts().head(15))

In [ ]:
# Industry distribution (adjust column name as needed)
industry_col = None
for col in df.columns:
    if 'industry' in col.lower() or 'sector' in col.lower():
        industry_col = col
        break

if industry_col:
    fig, ax = plt.subplots(figsize=(12, 8))
    industry_counts = df[industry_col].value_counts().head(20)
    industry_counts.plot(kind='barh', ax=ax, color='steelblue')
    ax.set_xlabel('Number of Companies')
    ax.set_title(f'Top 20 Industries ({industry_col})')
    plt.tight_layout()
    plt.show()
else:
    print("Industry column not found - please identify manually from column list")

## 6. Key Metrics Distribution

In [ ]:
# Find latest market cap column
mktcap_col = None
for col in df.columns:
    col_lower = col.lower()
    if ('market cap' in col_lower or 'marketcap' in col_lower) and 'latest' in col_lower:
        if '-' not in col_lower or 'latest]' in col_lower:
            mktcap_col = col
            break

# Find latest revenue column
revenue_col = None
for col in df.columns:
    col_lower = col.lower()
    if 'revenue' in col_lower and ('ltm]' in col_lower or 'latest' in col_lower):
        if '-' not in col_lower:
            revenue_col = col
            break

# Find employee column
emp_col = None
for col in df.columns:
    col_lower = col.lower()
    if 'employee' in col_lower and ('ltm]' in col_lower or 'latest' in col_lower):
        if '-' not in col_lower:
            emp_col = col
            break

print(f"Market Cap column: {mktcap_col}")
print(f"Revenue column: {revenue_col}")
print(f"Employee column: {emp_col}")

In [ ]:
# Distribution of key metrics
metrics_to_plot = [col for col in [mktcap_col, revenue_col, emp_col] if col is not None]

if metrics_to_plot:
    fig, axes = plt.subplots(1, len(metrics_to_plot), figsize=(5*len(metrics_to_plot), 4))
    if len(metrics_to_plot) == 1:
        axes = [axes]
    
    for ax, col in zip(axes, metrics_to_plot):
        data = df[col].dropna()
        # Use log scale for financial data
        data_positive = data[data > 0]
        if len(data_positive) > 0:
            np.log10(data_positive).hist(bins=50, ax=ax, color='steelblue', alpha=0.7)
            ax.set_xlabel(f'Log10({col[:30]}...)')
            ax.set_ylabel('Frequency')
            ax.set_title(f'Distribution (n={len(data_positive):,})')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Summary statistics for key metrics
if metrics_to_plot:
    print("\nSummary Statistics (in USD millions, except employees):")
    print("=" * 80)
    summary = df[metrics_to_plot].describe(percentiles=[.01, .05, .25, .5, .75, .95, .99])
    display(summary)

## 7. Research Feasibility Assessment

Based on the data exploration above, let's assess which research ideas are feasible.

In [ ]:
# Key dates for research design
KEY_EVENTS = {
    'ChatGPT Release': '2022-11-30',
    'COVID Pandemic': '2020-03-11',
    'Fed Rate Hikes Begin': '2022-03-16',
    'US-China Trade War Escalation': '2018-07-06',
    'SVB Collapse': '2023-03-10'
}

print("Key Events for Natural Experiments:")
print("=" * 60)
for event, date in KEY_EVENTS.items():
    print(f"{event}: {date}")

In [ ]:
def assess_research_feasibility(df, classified_cols):
    """
    Assess feasibility of different research questions based on data availability.
    """
    assessment = {}
    
    # 1. GenAI & Corporate Policy
    # Requires: R&D, CapEx, Employees, Market Cap, Industry classification
    genai_vars = ['expenses', 'cash_flow', 'other_metrics', 'market_cap', 'identifiers']
    genai_available = sum(1 for v in genai_vars if len(classified_cols.get(v, [])) > 0)
    assessment['GenAI & Corporate Policy'] = {
        'feasibility': 'HIGH' if genai_available >= 4 else 'MEDIUM' if genai_available >= 3 else 'LOW',
        'required_vars': genai_vars,
        'available': genai_available,
        'notes': 'Need R&D expense, CapEx, employee count, and industry to measure AI exposure'
    }
    
    # 2. COVID & Capital Structure
    # Requires: Debt, Cash, Total Assets, Profitability
    covid_vars = ['balance_sheet', 'profitability', 'ratios']
    covid_available = sum(1 for v in covid_vars if len(classified_cols.get(v, [])) > 0)
    assessment['COVID & Capital Structure'] = {
        'feasibility': 'HIGH' if covid_available == 3 else 'MEDIUM' if covid_available >= 2 else 'LOW',
        'required_vars': covid_vars,
        'available': covid_available,
        'notes': 'Strong data for leverage, liquidity, and profitability analysis'
    }
    
    # 3. Interest Rate Shock
    # Requires: Debt, Interest Expense, Investment (CapEx)
    rate_vars = ['balance_sheet', 'expenses', 'cash_flow']
    rate_available = sum(1 for v in rate_vars if len(classified_cols.get(v, [])) > 0)
    assessment['Interest Rate Shock (2022)'] = {
        'feasibility': 'HIGH' if rate_available == 3 else 'MEDIUM' if rate_available >= 2 else 'LOW',
        'required_vars': rate_vars,
        'available': rate_available,
        'notes': 'Interest expense and debt structure are key; can measure real effects via CapEx'
    }
    
    # 4. Labor vs Capital (Automation)
    # Requires: Employees, CapEx, Revenue, Industry
    labor_vars = ['other_metrics', 'cash_flow', 'identifiers']
    labor_available = sum(1 for v in labor_vars if len(classified_cols.get(v, [])) > 0)
    assessment['Labor-Capital Substitution'] = {
        'feasibility': 'HIGH' if labor_available == 3 else 'MEDIUM' if labor_available >= 2 else 'LOW',
        'required_vars': labor_vars,
        'available': labor_available,
        'notes': 'Employee count + CapEx + Revenue allows productivity analysis'
    }
    
    return assessment

feasibility = assess_research_feasibility(df, classified)

print("\n" + "=" * 80)
print("RESEARCH FEASIBILITY ASSESSMENT")
print("=" * 80)

for idea, details in feasibility.items():
    print(f"\n{idea}")
    print("-" * 40)
    print(f"  Feasibility: {details['feasibility']}")
    print(f"  Data coverage: {details['available']}/{len(details['required_vars'])} variable groups")
    print(f"  Notes: {details['notes']}")

## 8. Data Quality Checks for Panel Analysis

In [ ]:
# Check for duplicate companies
name_col = None
ticker_col = None

for col in df.columns:
    col_lower = col.lower()
    if 'name' in col_lower and name_col is None:
        name_col = col
    if 'ticker' in col_lower and ticker_col is None:
        ticker_col = col

if ticker_col:
    duplicates = df[ticker_col].duplicated().sum()
    print(f"Duplicate tickers: {duplicates}")
    if duplicates > 0:
        print("Duplicate examples:")
        dup_tickers = df[df[ticker_col].duplicated(keep=False)][ticker_col].unique()[:5]
        print(dup_tickers)

In [ ]:
# Check time series completeness for a sample metric
# Look for all columns related to a single metric across time

def check_time_series_completeness(df, metric_keyword):
    """Check how complete the time series is for a given metric."""
    cols = [c for c in df.columns if metric_keyword.lower() in c.lower()]
    
    if not cols:
        return None
    
    completeness = df[cols].notna().mean() * 100
    return completeness.sort_values(ascending=False)

# Check completeness for key metrics
for metric in ['revenue', 'market cap', 'employee', 'ebitda']:
    completeness = check_time_series_completeness(df, metric)
    if completeness is not None:
        print(f"\n{metric.upper()} - Time Series Completeness:")
        print(completeness.to_string())

## 9. Export Processed Data Structure

In [ ]:
# Create a data dictionary
data_dict = []

for col in df.columns:
    data_dict.append({
        'Column Name': col,
        'Data Type': str(df[col].dtype),
        'Non-Null Count': df[col].notna().sum(),
        'Null %': (df[col].isna().sum() / len(df)) * 100,
        'Unique Values': df[col].nunique(),
        'Sample Value': df[col].dropna().iloc[0] if df[col].notna().any() else None
    })

data_dict_df = pd.DataFrame(data_dict)
print(f"Data dictionary created with {len(data_dict_df)} columns")
display(data_dict_df.head(20))

In [ ]:
# Save data dictionary
data_dict_df.to_csv(DATA_PATH / 'data_dictionary.csv', index=False)
print(f"Data dictionary saved to {DATA_PATH / 'data_dictionary.csv'}")

## 10. Next Steps Summary

Based on this exploration, please review:

1. **Time coverage**: Does "Latest" refer to 2024? This determines which events you can study.

2. **Sample size**: Is the number of companies sufficient for your research design?

3. **Industry coverage**: Do you have enough variation in AI-exposed vs. non-exposed industries?

4. **Missing data patterns**: Are key variables available across your event window?

Run this notebook and share the outputs so we can proceed with the specific research design!

In [ ]:
# Final summary
print("\n" + "=" * 80)
print("EXPLORATION COMPLETE")
print("=" * 80)
print(f"\nTotal companies: {len(df):,}")
print(f"Total variables: {len(df.columns):,}")
print(f"\nPlease share the output of this notebook to proceed with research design.")